In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [9]:
data = pd.read_feather('database/GS.feather')

In [5]:
def deep_learning_dnn(df_dl, dep_var, classes):
    # Separa a variável dependente das demais
    deep_feat = df_dl.drop(columns=[dep_var], axis=1)
    deep_label = df_dl[dep_var]
    # Verifica os tipos das variáveis
    categorical_columns = [col for col in deep_feat.columns if len(deep_feat[col].unique()) == 2 or deep_feat[col].dtype == 'O']
    continuous_columns = [col for col in deep_feat.columns if len(deep_feat[col].unique()) > 2 and (deep_feat[col].dtype == 'int64' or deep_feat[col].dtype == 'float64')]
    # Verifica as colunas para normalização - as demais serão discretizadas - Função Bucketize do Tensor Flow
    cols_to_scale = continuous_columns[:]
    #cols_to_scale.remove('meses')
    # Ajusta as bases de treino e de teste
    XX_T = df_dl.drop(columns=[dep_var], axis=1)
    XX_t = df_dl.drop(columns=[dep_var], axis=1)
    yy_T = df_dl[dep_var]
    yy_t = df_dl[dep_var]
    # Normaliza as variáveis nas bases de treino e teste
    scaler = StandardScaler()
    XX_T.loc[:, cols_to_scale] = scaler.fit_transform(XX_T.loc[:, cols_to_scale]) ## Treinamento
    XX_t.loc[:, cols_to_scale] = scaler.fit_transform(XX_t.loc[:, cols_to_scale]) ## Teste
    # Ajustes das Variáveis Categórica - Não presentes neste modelo
    categorical_object_feat_cols = [tf.feature_column.embedding_column(
    tf.feature_column.categorical_column_with_hash_bucket(key=col, hash_bucket_size=1000), dimension=len(df_dl[col].unique()))
    for col in categorical_columns if df_dl[col].dtype == 'O']
    # Ajustes das Variáveis Categórica - Não presentes neste modelo
    categorical_integer_feat_cols = [tf.feature_column.embedding_column(
    tf.feature_column.categorical_column_with_identity(key=col, num_buckets=2), dimension=len(df_dl[col].unique()))
    for col in categorical_columns if df[col].dtype=='int64']
    continuous_feat_cols = [tf.feature_column.numeric_column(key=col) for col in continuous_columns] 
    feat_cols = categorical_object_feat_cols + \
                categorical_integer_feat_cols + \
                continuous_feat_cols
    # Rotina de DNN (Deep Neural Network)
    input_fun = tf.compat.v1.estimator.inputs.pandas_input_fn(XX_T, yy_T, batch_size=50, num_epochs=1000, shuffle=True)
    pred_input_fun = tf.compat.v1.estimator.inputs.pandas_input_fn(XX_t, batch_size=50, shuffle=False)
    DNN_model = tf.estimator.DNNClassifier(hidden_units=[10, 10, 10], feature_columns=feat_cols, n_classes=classes)
    DNN_model.train(input_fn=input_fun, steps=5000)
    # Resgata os resultados da DNN
    predictions = DNN_model.predict(pred_input_fun)
    pred = list(predictions)
    return pred

In [17]:
## Aula 14 - Probabilidade
dimensao = 'Country'
medidas = ['Sales', 'Quantity', 'Profit']
grupo = data.groupby(dimensao)[medidas].mean().reset_index()
grupo['Benefit'] = grupo['Profit'].apply(lambda x : 0 if x < 0 else 1)
grupo = grupo.set_index(dimensao)

In [18]:
grupo

,Sales,Quantity,Profit,Benefit
Country,,,,
Afghanistan,394.060364,4.145455,99.278182,1
Albania,243.007500,2.500000,44.332500,1
Algeria,184.140765,2.316327,46.461735,1
Angola,209.459016,2.598361,53.237459,1
Argentina,147.466111,3.856410,-47.932812,0
...,...,...,...,...
Venezuela,137.050668,3.989691,-57.849023,0
Vietnam,248.302639,3.758491,-7.057474,0
Yemen,82.190400,2.366667,-123.548600,0


In [20]:
probabilidade = deep_learning_dnn(grupo, 'Benefit', 2)
probabilidade_classe = []
for i in range(len(probabilidade)):
    probabilidade_classe.append(probabilidade[i]["class_ids"][0])
probabilidade_prob0 = []
for i in range(len(probabilidade)):
    probabilidade_prob0.append(probabilidade[i]["probabilities"][0])
probabilidade_prob1 = []
for i in range(len(probabilidade)):
    probabilidade_prob1.append(probabilidade[i]["probabilities"][1]) 
grupo['dl_classe'] = probabilidade_classe
grupo['lucro_0'] = probabilidade_prob0
grupo['lucro_1'] = probabilidade_prob1
grupo = grupo.reset_index()
grupo

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp951ji61f', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:ten

,Country,Sales,Quantity,Profit,Benefit,dl_classe,lucro_0,lucro_1
0,Afghanistan,394.060364,4.145455,99.278182,1,1,0.473232,0.526768
1,Albania,243.007500,2.500000,44.332500,1,1,0.472677,0.527323
2,Algeria,184.140765,2.316327,46.461735,1,1,0.472606,0.527394
3,Angola,209.459016,2.598361,53.237459,1,1,0.473225,0.526775
4,Argentina,147.466111,3.856410,-47.932812,0,1,0.489815,0.510185
...,...,...,...,...,...,...,...,...
142,Venezuela,137.050668,3.989691,-57.849023,0,1,0.493513,0.506487
143,Vietnam,248.302639,3.758491,-7.057474,0,1,0.478172,0.521828
144,Yemen,82.190400,2.366667,-123.548600,0,1,0.492406,0.507594
145,Zambia,255.250000,2.460784,68.644412,1,1,0.472063,0.527937


In [15]:
grupo.to_feather('database/probabilidade_pais.feather')